#### Imports

In [1]:
# Telegram Bot
import telebot
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

# Config Driver
from config.config_driver import ConfigDriver

# Amazon Driver
from amazon.amazon import Amazon

# Definitions
from definitions import *

#### Variables Main

In [2]:
API_TOKEN = 'Your_Token_Here'
bot = telebot.TeleBot(API_TOKEN)

config_driver = ConfigDriver()
driver = config_driver.get_driver()

amazon_config = Amazon(driver)

#### Main

In [3]:

def get_function_details(message):
    """ Method to get the function details """
    
    response = {}
    
    if message.text == '/start':
        response['message'] = "Assistente Virtual da Amazon inicializado!"
    
    elif message.text in SAUDACOES:
        user = message.from_user.first_name
        
        markup = InlineKeyboardMarkup()
        option1 = InlineKeyboardButton("Pesquisar Preço de Produto", callback_data="opt1")
        option2 = InlineKeyboardButton("Ofertas do Dia", callback_data="opt2")
        option3 = InlineKeyboardButton("Prime Vídeo", callback_data="opt3")
        response['markup'] = markup.add(option1, option2, option3)
        
        response['text'] = f"Eae {user}! Em que posso te ajudar?"
    else:
        response['message'] = "Desculpe, eu não entendi o que você disse."
        
    return response

################################################################################
@bot.message_handler(func=lambda message: True)
def reply_message(message):
    """ Method to reply the message """
    
    response = get_function_details(message)
    
    if 'markup' in response:
        bot.reply_to(message, response['text'], reply_markup=response["markup"])
    elif 'message' in response:
        bot.send_message(message.chat.id, response['message'])
    else:
        bot.reply_to(message, response['text'])

################################################################################
@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    """ Method to handle the callback query """
    
    if call.data == "opt1":
        bot.answer_callback_query(call.id, "Pesquisar Preço de Produto!")
    
    elif call.data == "opt2":
        bot.answer_callback_query(call.id, "Ofertas do Dia!")
        
        departments = amazon_config.get_departaments_offer_of_the_day()
        
        bot.send_message(call.message.chat.id, f"Departamentos disponíveis:\n{departments}")
        bot.send_message(call.message.chat.id, "Qual departamento você deseja ver as ofertas?")
    
    elif call.data == "opt3":
        bot.answer_callback_query(call.id, "Prime Vídeo!")

################################################################################

#### Run Bot

In [4]:
bot.polling()